In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch.nn as nn
import math
import torch.optim as optim

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

# This SEModule is not used.
class SEModule(nn.Module):

    def __init__(self, planes, compress_rate):
        super(SEModule, self).__init__()
        self.conv1 = nn.Conv2d(planes, planes // compress_rate, kernel_size=1, stride=1, bias=True)
        self.conv2 = nn.Conv2d(planes // compress_rate, planes, kernel_size=1, stride=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = F.avg_pool2d(module_input, kernel_size=module_input.size(2))
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.sigmoid(x)
        return module_input * x


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class SENet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, include_top=True):
        self.inplanes = 64
        super(SENet, self).__init__()
        self.include_top = include_top
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, ceil_mode=True)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        
        if not self.include_top:
            return x
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def senet18(num_classes):
    model = SENet(BasicBlock, [2, 2, 2, 2],num_classes=num_classes)
    return model

In [3]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

#建立自己的dataset train_data
class MyDataset(Dataset):
    def __init__(self, image_path: list, image_class: list, transform=None):
        self.image_path = image_path
        self.image_class = image_class
        self.transform = transform

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, item):
        img = Image.open(self.image_path[item]).convert('RGB')
#         if img.mode != 'RGB':
#             raise ValueError("image: {} isn't RGB mode".format(self.image_path[item]))
        label = self.image_class[item]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [4]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

epoch_num=20
batch_size=128
lr=0.0003
path_photo='/kaggle/input/bitmojidataset/trainimages'
path_label='/kaggle/input/bitmojidataset/train.csv'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

# temp_name = os.listdir(path_photo)
df=pd.read_csv(path_label)
image_path=[]
label_path = []
for dir in os.listdir(path_photo):
#     print(dir)
    image_path.append(path_photo+'/'+str(dir))
    # 得到所有图片label
    if df[df.image_id == dir].iloc[0, 1]==-1:
        label_path.append(0)
    else:
        label_path.append(df[df.image_id == dir].iloc[0, 1])

image_path=np.array(image_path)
label_path=np.array(label_path)

train_path,val_path,train_label,val_label=train_test_split(image_path,label_path,test_size=0.2,random_state=1)
# print(len(train_path),len(val_path))
train_data=MyDataset(train_path,train_label,transform)
val_data=MyDataset(val_path,val_label,transform)
train_loader = DataLoader(train_data, batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size,shuffle=False)
print(train_loader)
print(val_loader)

In [5]:
net=senet18(num_classes=2)
if torch.cuda.is_available():
    net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(), lr=lr)
# 参数量
n_p=sum(x.numel() for x in net.parameters())
print(n_p)

11177538


In [7]:
# 训练
import time
# iter=0
loss=0
acc=0
n=0
print('start training!')
start=time.time()
for epoch in range(epoch_num):
    for img,label in train_loader:
        # 将数据移到GPU上
        img=img.cuda()
        label=label.cuda()
        # 先将optimizer梯度先置为0
        optimizer.zero_grad()
        out=net(img)
        # 计算loss,图的终点处
        loss0=criterion(out,label)
        loss=loss0

#         反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        prect =out.argmax(1)
        num_correct=(prect==label).sum()
        acc+=num_correct.item()
        n+=label.shape[0]
    print('epoch: {}, loss: {:.4} Acc: {:.6f}'.format(epoch, loss.data.item(),acc / n))

start= time.time()-start
print(start)
print("train over!")
if not os.path.exists('./model_save'):
    os.mkdir('./model_save')
torch.save(net.state_dict,os.path.join('./model_save/','epoch_{}_.pth'.format(epoch+1)))

start training!


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


epoch: 0, loss: 0.04438 Acc: 0.928750
epoch: 1, loss: 0.007823 Acc: 0.961042
epoch: 2, loss: 0.0007669 Acc: 0.973611
epoch: 3, loss: 0.0005692 Acc: 0.980208
epoch: 4, loss: 0.002044 Acc: 0.984083
epoch: 5, loss: 0.0009296 Acc: 0.986667
epoch: 6, loss: 0.0002263 Acc: 0.988571
epoch: 7, loss: 0.0001589 Acc: 0.990000
epoch: 8, loss: 0.0001299 Acc: 0.991111
epoch: 9, loss: 0.0001098 Acc: 0.992000
epoch: 10, loss: 9.57e-05 Acc: 0.992727
epoch: 11, loss: 8.496e-05 Acc: 0.993333
epoch: 12, loss: 7.635e-05 Acc: 0.993846
epoch: 13, loss: 6.92e-05 Acc: 0.994286
epoch: 14, loss: 6.317e-05 Acc: 0.994667
epoch: 15, loss: 5.801e-05 Acc: 0.995000
epoch: 16, loss: 5.353e-05 Acc: 0.995294
epoch: 17, loss: 4.962e-05 Acc: 0.995556
epoch: 18, loss: 4.616e-05 Acc: 0.995789
epoch: 19, loss: 4.311e-05 Acc: 0.996000
340.3100345134735
train over!


In [8]:
import torch
# 模型评估
net.eval()
eval_loss=0
eval_acc=0
for img,label in val_loader:
    if torch.cuda.is_available():
        img=img.cuda()
        label=label.cuda()
    out=net(img)
    loss=criterion(out,label)
    eval_loss+=loss.item()*label.size(0)
    prect =out.argmax(1)
    num_correct=(prect==label).sum()
    eval_acc+=num_correct.item()
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(val_data)), eval_acc / (len(val_data))))

    

Test Loss: 0.014680, Acc: 0.995000


In [10]:
# 测试
import torch
import pandas as pd
testimg_path='/kaggle/input/bitmojidataset/testimages'
submission_path='/kaggle/input/bitmojidataset/sample_submission.csv'

cnt=0
test_df=pd.read_csv(submission_path)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

# net.eval()
with torch.no_grad():
    parent_list=os.listdir(testimg_path)
    parent_list.sort()
#     print(parent_list)
    for img_name in parent_list:
        img_path = os.path.join(testimg_path, img_name)
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        if torch.cuda.is_available():
            img=img.cuda()
        img = img.unsqueeze(0)
        prect = int(net(img).argmax(1))
        if prect==0:
            prect=-1
#         print(prect)
        test_df['is_male'].iloc[cnt:cnt+1]=str(prect)
        cnt+=1
# test_df.iloc[:,1:2]
print(test_df)    
test_df.to_csv('./sample_submission.csv',index=False,header=True)
print('test_over!')

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


      image_id is_male  left_eye_x  left_eye_y  right_eye_x  right_eye_y  \
0     3000.jpg      -1       152.0       198.0        233.0        195.0   
1     3001.jpg      -1         NaN         NaN          NaN          NaN   
2     3002.jpg      -1         NaN         NaN          NaN          NaN   
3     3003.jpg       1         NaN         NaN          NaN          NaN   
4     3004.jpg      -1         NaN         NaN          NaN          NaN   
...        ...     ...         ...         ...          ...          ...   
1079  4079.jpg      -1         NaN         NaN          NaN          NaN   
1080  4080.jpg       1         NaN         NaN          NaN          NaN   
1081  4081.jpg       1         NaN         NaN          NaN          NaN   
1082  4082.jpg      -1         NaN         NaN          NaN          NaN   
1083  4083.jpg       1         NaN         NaN          NaN          NaN   

      nose_x  nose_y  mouth_left_x  mouth_left_y  mouth_right_x  mouth_right_y  
0     